In [7]:
import pandas as pd
import numpy as np
import logging
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from pathlib import Path
import itertools

from config import shuffled_csv, path_exps
from NN import NN_model, Sigmoid, MSE, L2_reg
from NN.utility import batch_train, batch_out, Model_Wrapper
from LBFGS import LBFGS
from testing import multi_run


In [9]:
data = pd.read_csv(shuffled_csv, index_col=0).to_numpy()
data = data[:10, :]
n_samples = data.shape[0]
X_data = data[:, :10]
Y_data = data[:, 10:]

Y_scaler = StandardScaler()
Y_scaled = Y_scaler.fit_transform(Y_data)

# np.random.seed(11)
model = NN_model([10, 20, 20, 2], Sigmoid, MSE)
model.init_weights()
reg_loss = L2_reg(1e-4)

# logging.basicConfig(level="INFO")
f = Model_Wrapper(model, X_data, Y_scaled, reg_loss)
x = model.Weights

##########################################
m1 = [1e-4, 1e-3, 1e-2]
m2 = [0.9, 0.7, 0.5]
HPs = itertools.product(m1, m2)

experiments_csv = path_exps / Path("LBFGS_grid_search.csv")
# remove HP combinations already explored
if experiments_csv.exists():
    df = pd.read_csv(experiments_csv)
    HPs_new = []
    for m1,m2 in HPs:
        if ((df['m1'] == m1) & (df['m2'] == m2)).any():
            continue
        else:
            HPs_new.append((m1,m2))
    HPs = HPs_new

for m1, m2 in HPs:
    print(f'Testing m1: {m1}, m2: {m2}', end='')
    solver = LBFGS(eps=1e-4, max_feval=5e4, M=10,m1=m1,m2=m2)
    results = multi_run(solver, f, "LBFGS", n=2)

    log_columns = ["m1", "m2", "f_evals_m", "f_evals_std", "n_failures"]
    log_data = [m1, m2, results[2], results[3], results[6]]
    df = pd.DataFrame([log_data], columns=log_columns)

    if experiments_csv.exists():
        df.to_csv(experiments_csv, mode="a", header=False,index=False)
    else:
        df.to_csv(experiments_csv,index=False)
    print('\t Finished')


Testing m1: 0.0001, m2: 0.9

TypeError: multi_run() got multiple values for argument 'n'

In [6]:
df = pd.read_csv(experiments_csv)
df.sort_values(by=['f_evals_m'])

,m1,m2,f_evals_m,f_evals_std,n_failures
1,0.0001,0.7,732.0,244.0,0
4,0.0010,0.7,732.0,244.0,0
7,0.0100,0.7,732.0,244.0,0
0,0.0001,0.9,1134.0,378.0,0
3,0.0010,0.9,1134.0,378.0,0
6,0.0100,0.9,1134.0,378.0,0
2,0.0001,0.5,NaN,NaN,2
5,0.0010,0.5,NaN,NaN,2
8,0.0100,0.5,NaN,NaN,2
